# Calculate Angular Momentum 

Adapted from Neil's measure_halo_spin.ipynb. Calculates the unit angular momentum axes for a tilted-ring model of the disk for  halos in the halo catalogue and which have supplementary circularity data. Additional code for bootstrapping.

### Initial Setup

In [4]:
import sys
import h5py
import os
from os.path import exists
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from numpy.linalg import inv
from scipy.spatial.transform import Rotation as R

from FigureRotation_util import *
from astropy import constants as const
from astropy import units as u
from tqdm import tqdm

sys.path.append('/home/tnguser/python/')

In [5]:
sim = 'L35n2160TNG' # simulation name
startSnap = 75 # starting snapshot for principal & pattern files
snapIdx = 24 # index of most recent snapshot (z=0)
basePath = '/home/tnguser/sims.TNG/' + sim + '/output'

# angular momentum axis of stars in 3kpc sphere
spinPath = '/home/tnguser/postprocessing/angular_momentum/' + sim + '/00_03kpc/'
# halo principal axes
principal_path =  '/home/tnguser/postprocessing/principal_axes/' + sim + '/00_06Rvir/'
# figure rotation axes
pattern_path =  '/home/tnguser/postprocessing/pattern_speeds/' + sim + '/00_06Rvir/'

# getting IDs of halos in halo catalogue which are "disky"
filename = "./stellar_circs.hdf5" # supplementary stellar circularity information
hf=h5py.File(filename,'r')
circ_subfindID = hf['Snapshot_99']['SubfindID']
circ07 = hf['Snapshot_99']['CircAbove07Frac']
disky_temp = circ_subfindID[np.where(np.array(circ07) > 0.5)] # definition of "disky" halos
main_subhalos = np.load('/home/tnguser/postprocessing/halocatalogues/' + sim + '_mainSubhalos.npy') #subfindIDs of halos in Neil's catalogue
diskyIDs = np.intersect1d(disky_temp,main_subhalos) # final set of subfindIDs

### Helper Functions

In [6]:
# Taken from Neil's measure_halo_spin.ipynb
def snapshot_redshift_corr(basePath,startSnap=75):
    redshift_space = []
    for snapshot_number in range(startSnap,100):
        header=il.groupcat.loadHeader(basePath,snapshot_number)
        redshift_space.append(header.get('Redshift'))
    return np.arange(startSnap,100), np.array(redshift_space)

# Taken from Neil's measure_halo_spin.ipynb
def value(arr):
    """
    Convenience function, returns the item contained by an array,
    or an error if the array has more than one element
    """
    assert len(arr)<2
    return arr[0]


def make_unit(vec):
    """
    Convenience function, returns the unit-normalized vector
    """
    return 1/np.sqrt(np.sum(vec**2)) * vec


def getRotationAxis(snap, startStops, Raxis):
    """
    Returns the figure rotation axis at the snapshot snap
    """
    for idx, (first,second) in enumerate(startStops):
        if (snap >= first and snap < second):
            return Raxis[idx]
    return Raxis[-1]


def angleBtwn(u,v):
    """
    Returns angle between vectors u and v
    """
    u = np.array(u)
    v= np.array(v)
    u = 1/np.sqrt(np.sum(u**2))*u
    v = 1/np.sqrt(np.sum(v**2))*v
    
    return np.arccos(np.dot(u,v))

def getAngles(v):
    """
    Returns the Brigg's angles in form [polar angle, azmuthal angle]
    """
    polar_angle = angleBtwn(v,[0,0,1])
    xy_angle = np.sign(v[1])*np.arccos(v[0]/np.sqrt(v[0]**2+v[1]**2))
    return [polar_angle, xy_angle]

## Angular Momentum Calculation

In [11]:
### CHOOSE BINS FOR CALCULATION ************************************************
tilted_rings = True
log_spacing = True

if (tilted_rings):
    if (log_spacing): # rings are evenly (ish) spaced in log-scale
        lims = [[0,0.647], [0.647,2.212], [2.212, 4.572], [4.572,7.815], [7.815,15]]
        save_name = '/home/tnguser/postprocessing/angular_momentum/' + sim + '/rings_log/'
    else: # rings are evenly spaced in linear-scale
        lims = [[0,3],[3,6],[6,9],[9,12],[12,15]]
        save_name = '/home/tnguser/postprocessing/angular_momentum/' + sim + '/rings_linear/'
else: # set one inner radii and outer raddi [kpc] lims
    innerLim = 0.0
    outerLim = 3.0
    lims = [[innerLim, outerLim]]
    save_name = '/home/tnguser/postprocessing/angular_momentum/' + sim + '/%02d_%02dkpc/'%(10*innerLim,outerLim)

In [ ]:
if not exists(save_name):
        os.mkdir(save_name)

total = len(diskyIDs)
num_completed = 1
for subfindID in diskyIDs:
    
    print("working on #", num_completed, "of ", total)
    
    ### CODE IN COMMENTS ADAPTED FROM NEIL'S "measure_halo_spin.py" ----------------------------------------------
    ### ----------------------------------------------------------------------------------------------------------
    GrNr = (il.groupcat.loadSingle(basePath, 99, subhaloID=subfindID)['SubhaloGrNr'])
    
    snapArr,zArr = snapshot_redshift_corr(basePath)

    haloTree = il.lhalotree.loadTree(basePath,99,subfindID,
                                     fields=['SubhaloGrNr','SnapNum','Group_R_Crit200','SubhaloPos','SubhaloNumber'], 
                                     onlyMPB=True)

    haloInd,mpb_snapArr = haloTree['SubhaloGrNr'], haloTree['SnapNum']
    subfindID = haloTree['SubhaloNumber']
    Rvirial   = haloTree['Group_R_Crit200']
    subhaloPos = haloTree['SubhaloPos']

    snap = 99 # only calculating angular momentum for most recent snap
    GrNr_i       = value(haloInd[snap == mpb_snapArr])
    subfindID_i  = value(subfindID[snap == mpb_snapArr])
    Rvirial_i    = value(Rvirial[snap == mpb_snapArr])
    subhaloPos_i = value(subhaloPos[snap == mpb_snapArr])
    a = 1/(1+value(zArr[snap==snapArr]))

    starPos = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Coordinates')
    starVel = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Velocities')
    starMass = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Masses') 

    # following Neil's method, but only using stars (not gas or DM)
    if (type(starPos) is not dict):
        particleCoords_bar  = starPos
        particleVels_bar    = starVel
        particleMass_bar    = starMass * 1e10
    else:
        # no stars in the halo
        print("ERROR: no stars in halo.")
        particleCoords_bar  = np.array([[np.nan,np.nan,np.nan]])
        particleVels_bar    = np.array([[np.nan,np.nan,np.nan]])
        particleMass_bar    = np.array([np.nan])

    ########################################################################
    # check that the halo is not crossing the box boundary, correct if it is
    # condition is whether it is within 2 virial radii
    passUpperBound = subhaloPos_i + 2*Rvirial_i >= 35000
    passLowerBound = subhaloPos_i - 2*Rvirial_i <= 0

    if any(passUpperBound):
        # shift the particles
        particleCoords_bar[:,passUpperBound] = particleCoords_bar[:,passUpperBound] - 35000/2
        subhaloPos_i[passUpperBound] = subhaloPos_i[passUpperBound] - 35000/2
        #reinforce boundary condition
        ind_bar = np.where(particleCoords_bar[:,passUpperBound]<0)[0]
        particleCoords_bar[ind_bar,passUpperBound] = particleCoords_bar[ind_bar,passUpperBound] + 35000

    if any(passLowerBound):
        particleCoords_bar[:,passLowerBound] = particleCoords_bar[:,passLowerBound] + 35000/2 
        subhaloPos_i[passLowerBound] = subhaloPos_i[passLowerBound] + 35000/2

        ind_bar = np.where(particleCoords_bar[:,passLowerBound]>35000)[0]
        particleCoords_bar[ind_bar,passLowerBound] = particleCoords_bar[ind_bar,passLowerBound] - 35000
    ########################################################################

    # center coords on subhalo pos
    particleCoords_bar = (particleCoords_bar - subhaloPos_i)*a
    # find relative velocities - shouldn't have done this, instead use the spin by TNG (includes DM)
    paricleVels_bar = (particleVels_bar - np.mean(particleVels_bar,axis=0)) * np.sqrt(a) # km/s
    
    ### end Neil's code  -----------------------------------------------------------------------------------------
    ### ----------------------------------------------------------------------------------------------------------

    # angular momentum of stars in inner 3kpc sphere - used as perpendicular to stellar disk
    spin_file = spinPath + 'GrNr_%d_snap_%d_99_lambda_prime_baryons.npy'%(GrNr,99)
    spin = np.load(spin_file,allow_pickle=True)[-1]  # gets most recent spin
    spin = make_unit(spin)
    
    all_spins = []

    for innerRad, outerRad in lims:
        ### BOUNDARY CONDITIONS FOR DISK STARS: must be located in an annulus of +- 1 kpc height with radius
        ### between innter & outer radii as specified above
    
        perp = spin * np.expand_dims(np.dot(particleCoords_bar,spin),axis=1) # projection of star position onto disk perpendicular
        parallel = particleCoords_bar - perp # component of star position parallel to disk 

        indsInBounds = np.where((np.sqrt(np.sum(perp**2,axis=1)) <= 1.0) & (np.sqrt(np.sum(parallel**2,axis=1)) <= outerRad) & (np.sqrt(np.sum(parallel**2,axis=1)) > innerRad))[0]

        J_bar = np.sum(particleMass_bar[indsInBounds].reshape(-1,1)*np.cross(particleCoords_bar[indsInBounds],
                                                                            particleVels_bar[indsInBounds]),axis=0) # measured in units of h^{-2} Msun*kpc*km/s
        all_spins.append(make_unit(J_bar))

    temp = save_name+'GrNr_%d_snap_%d_99_lambda_prime_baryons.npy'%(GrNr,99)
    np.save(temp,all_spins,allow_pickle=True)
    
    num_completed = num_completed + 1

## Bootstrapping

Bootstraps the above method for calculating angular momentum. Chooses "P" percent of stars with repetition for "N" iterations. Calculates the briggs angles of each angular momentum ring vector "N" times. *MUST SET VARIABLE "lims" ABOVE CORRESPONDING TO TILTED-RING RADII

In [9]:
# Helper funcation which calculates Briggs angles for each angular momentum axis in array spins

def getBriggsAngles(spins, GrNr, subfindID):
    
    principal_file = principal_path + 'GrNr_%d_snap_%d_99_principal_axes_full.npy'%(GrNr,75)
    pattern_file = pattern_path + 'GrNr_%d_snap_%d_99_patternSpeeds.npy'%(GrNr,75)
    
    theta = []; phi = []
    if exists(principal_file) and exists(pattern_file):
        
        principal_axes = np.load(principal_file,allow_pickle=True)
        pattern_info = np.load(pattern_file, allow_pickle=True)
        pattern_info = dict(enumerate(pattern_info.flatten(),1))[1]
        
        raxis = getRotationAxis(25, pattern_info['startStop'], pattern_info['Raxis'])
        raxis = 1/np.sqrt(np.sum(raxis**2))*raxis
        
        # Finds transformation matrix that rotates figure rotation axis to align with z-axis, tries to align
        # halo major axis with x-axis
        z = [[0, 0, 1], [1, 0, 0]]
        rot, rssd, sens = R.align_vectors(z, [raxis, [1,0,0]], return_sensitivity=True, weights=[100, .1])
        
        for spin in spins:
            spin_bf = np.dot(inv(principal_axes[snapIdx]), spin) # Changes to body frame coordinates
            spin_new = (rot.apply(spin_bf)) # Then applies new rotation 
            spin_new = 1/np.sqrt(np.sum(spin_new**2))*spin_new
                
            # Theta (polar angle): angle between fiducial z-axis (figure rotation axis)
            # Phi (azimuthal angle): angle between projection onto xy-plane and x-axis (near halo major axis)
            t,p = getAngles(spin_new)
            theta.append(t)
            phi.append(p)
            
    return theta, phi

In [ ]:
p = .90  # percent of stars chosen
N = 100  # number of bootstrap iterations 
IDs = [672013] # subfindIDs of halos to bootstrap

diskPath = '/home/tnguser/postprocessing/angular_momentum/' + sim + '/00_03kpc/' # Vector perpendicular to disk

for subfindID in IDs:
        
    GrNr = (il.groupcat.loadSingle(basePath, 99, subhaloID=subfindID)['SubhaloGrNr'])
    
    # Vector perpendicular to disk
    disk_file = diskPath + 'GrNr_%d_snap_%d_99_lambda_prime_baryons.npy'%(GrNr,99)
    disk = np.load(disk_file,allow_pickle=True)[-1]
    disk = make_unit(disk)
    
    # Same as above ------------------------------------------------------------------------------------------
    snapArr,zArr = snapshot_redshift_corr(basePath)

    haloTree = il.lhalotree.loadTree(basePath,99,subfindID,
                                     fields=['SubhaloGrNr','SnapNum','Group_R_Crit200','SubhaloPos','SubhaloNumber'], 
                                     onlyMPB=True)

    haloInd,mpb_snapArr = haloTree['SubhaloGrNr'], haloTree['SnapNum']
    subfindID = haloTree['SubhaloNumber']
    Rvirial   = haloTree['Group_R_Crit200']
    subhaloPos = haloTree['SubhaloPos']

    snap = 99
    GrNr_i       = value(haloInd[snap == mpb_snapArr])
    subfindID_i  = value(subfindID[snap == mpb_snapArr])
    Rvirial_i    = value(Rvirial[snap == mpb_snapArr])
    subhaloPos_i = value(subhaloPos[snap == mpb_snapArr])
    a = 1/(1+value(zArr[snap==snapArr]))

    starPos = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Coordinates')
    starVel = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Velocities')
    starMass = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Masses') 

    particleCoords_bar  = starPos
    particleVels_bar    = starVel
    particleMass_bar    = starMass * 1e10

    ########################################################################
    # check that the halo is not crossing the box boundary, correct if it is
    # condition is whether it is within 2 virial radii
    passUpperBound = subhaloPos_i + 2*Rvirial_i >= 35000
    passLowerBound = subhaloPos_i - 2*Rvirial_i <= 0

    if any(passUpperBound):
        print("Shifted - up")
        # shift the particles
        particleCoords_bar[:,passUpperBound] = particleCoords_bar[:,passUpperBound] - 35000/2
        subhaloPos_i[passUpperBound] = subhaloPos_i[passUpperBound] - 35000/2
        #reinforce boundary condition
        ind_bar = np.where(particleCoords_bar[:,passUpperBound]<0)[0]
        particleCoords_bar[ind_bar,passUpperBound] = particleCoords_bar[ind_bar,passUpperBound] + 35000

    if any(passLowerBound):
        print("Shifted - down")
        particleCoords_bar[:,passLowerBound] = particleCoords_bar[:,passLowerBound] + 35000/2 
        subhaloPos_i[passLowerBound] = subhaloPos_i[passLowerBound] + 35000/2

        ind_bar = np.where(particleCoords_bar[:,passLowerBound]>35000)[0]
        particleCoords_bar[ind_bar,passLowerBound] = particleCoords_bar[ind_bar,passLowerBound] - 35000
    ########################################################################

    # center coords on subhalo pos
    particleCoords_bar = (particleCoords_bar - subhaloPos_i)*a
    # find relative velocities - shouldn't have done this, instead use the spin by TNG (includes DM)
    paricleVels_bar = (particleVels_bar - np.mean(particleVels_bar,axis=0)) * np.sqrt(a) # km/s
    
    # -------------------------------------------------------------------------------------------------------

    # BOOTSTRAPPING HERE
    spins_theta = []
    spins_phi = []
    
    for i in np.arange(N):
        print("Working on bootstrap ", i+1, " of ", N)
        spins = []
        
        for innerRad, outerRad in lims: ### SET LIMS IN CODE BLOCK ABOVE ANGULAR MOMENTUM CALCULATION
            
            perp = spin * np.expand_dims(np.dot(particleCoords_bar,spin),axis=1) 
            parallel = particleCoords_bar - perp
            indsInBounds = np.where((np.sqrt(np.sum(perp**2,axis=1)) <= 1.0) & (np.sqrt(np.sum(parallel**2,axis=1)) <= outerRad) & (np.sqrt(np.sum(parallel**2,axis=1)) > innerRad))[0]
            
            # Choosing p percent of samples with repetition
            num_data = len(indsInBounds)
            num_samples = int(p * num_data)
            if i == 0:
                print("Using ", num_samples, " of ", num_data) # Want these to be ~equal
            random_indices = (np.random.randint(0,num_data,size=num_samples))
            bootstrap_inds = np.array(indsInBounds)[random_indices.astype(int)]
            
            J_bar = np.sum(particleMass_bar[bootstrap_inds].reshape(-1,1)*np.cross(particleCoords_bar[bootstrap_inds],
                                                                                particleVels_bar[bootstrap_inds]),axis=0) 
            spins.append(make_unit(J_bar))
            
        theta, phi = getBriggsAngles(spins, GrNr, subfindID)
        spins_theta.append(theta)
        spins_phi.append(phi)
        
    theta_mean = np.mean(spins_theta,axis=0)
    phi_mean = np.mean(spins_phi,axis=0)
    
    theta_std = np.std(spins_theta,axis=0)
    phi_std = np.std(spins_phi,axis=0)
    

In [ ]:
# OLD VERSION THAT STILL CALCULATED FOR DARK MATTER SPIN ALSO

sim = 'L35n2160TNG'

# might want to change these ** 
innerVirialLim = 0
outerVirialLim = 0.6
# ** 

basePath = '/home/tnguser/sims.TNG/' + sim + '/output'
catalogue_path = '/home/tnguser/postprocessing/halocatalogues/' + sim + '.npy'
naive_halos = np.load(catalogue_path)
main_subhalos = np.load('/home/tnguser/postprocessing/halocatalogues/' + sim + '_mainSubhalos.npy') 
save_name = '/home/tnguser/postprocessing/spin/' + sim + '/%02d_%02dRvir/'%(10*innerVirialLim,10*outerVirialLim)
    
if not os.path.exists(save_name):
    os.mkdir(save_name)

working_dir = os.getcwd()

startSnap = 99
rnorm = False

restart = True
if restart:
    if os.path.exists(sim+'_halo_spin_restart.npy'):
        completed_halos = np.load(sim+'_halo_spin_restart.npy')
    else:
        completed_halos = []

print('getting redshift/snapshot relation, particle mass...')

#h0 = 0.6774
snapArr,zArr = snapshot_redshift_corr(basePath)
mDM = np.load('/home/tnguser/python/'+sim+'_mDM.npy').item() 
DM = 1
print('done')
    
finishedCount = 0
for GrNr,subfindID in zip(naive_halos,main_subhalos):
        
    if GrNr == -1:
        finishedCount += 1
        continue

    if restart and (GrNr in completed_halos):
        finishedCount += 1
        continue

    print("Beginning work on halo %d (%d/%d complete)"%(GrNr,finishedCount,len(naive_halos)))
    finishedCount+= 1

        
    #subfindID = groupFirstSub_99[GrNr]
    haloTree = il.lhalotree.loadTree(basePath,99,subfindID,
                                        fields=['SubhaloGrNr','SnapNum','Group_R_Crit200','SubhaloPos','SubhaloNumber'], 
                                        onlyMPB=True)
    haloInd,mpb_snapArr = haloTree['SubhaloGrNr'], haloTree['SnapNum']
    subfindID = haloTree['SubhaloNumber']
    Rvirial   = haloTree['Group_R_Crit200']
    subhaloPos = haloTree['SubhaloPos']
        
    lambda_mag  = []
    lambda_dir  = []
    lambda_mag_bar = []
    lambda_dir_bar = []
    for snap in tqdm(range(startSnap,100)):

        GrNr_i       = value(haloInd[snap == mpb_snapArr])
        subfindID_i  = value(subfindID[snap == mpb_snapArr])
        Rvirial_i    = value(Rvirial[snap == mpb_snapArr])
        subhaloPos_i = value(subhaloPos[snap == mpb_snapArr])
            
        particleCoords = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,DM,fields='Coordinates')
        particleVels   = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,DM,fields='Velocities')
            
        if 'DM' not in sim:
            starPos = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Coordinates')
            gasPos  = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,0,fields='Coordinates')
                
            starVel = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Velocities')
            gasVel  = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,0,fields='Velocities')
                
            starMass = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,4,fields='Masses') 
            gasMass = il.snapshot.loadSubhalo(basePath,snap,subfindID_i,0,fields='Masses') 
                
                
            if (type(starPos) is not dict) and (type(gasPos) is not dict):
                particleCoords_bar  = np.concatenate((starPos,gasPos),axis=0)
                particleVels_bar    = np.concatenate((starVel,gasVel),axis=0)
                particleMass_bar    = np.concatenate((starMass,gasMass),axis=0) * 1e10
            elif (type(starPos) is dict) and (type(gasPos) is not dict):
                particleCoords_bar  = gasPos
                particleVels_bar    = gasVel
                particleMass_bar    = gasMass * 1e10
            elif (type(starPos) is not dict) and (type(gasPos) is dict):
                particleCoords_bar  = starPos
                particleVels_bar    = starVel
                particleMass_bar    = starMass * 1e10
            else:
                # no baryons in the halo
                particleCoords_bar  = np.array([[np.nan,np.nan,np.nan]])
                particleVels_bar    = np.array([[np.nan,np.nan,np.nan]])
                particleMass_bar    = np.array([np.nan])
                    
            
        a = 1/(1+value(zArr[snap==snapArr]))
            
        ########################################################################
        # check that the halo is not crossing the box boundary, correct if it is
        # condition is whether it is within 2 virial radii
        passUpperBound = subhaloPos_i + 2*Rvirial_i >= 35000
        passLowerBound = subhaloPos_i - 2*Rvirial_i <= 0
        if any(passUpperBound):
            #shift the particles
            particleCoords[:,passUpperBound] = particleCoords[:,passUpperBound] - 35000/2
            subhaloPos_i[passUpperBound] = subhaloPos_i[passUpperBound] - 35000/2
            #reenforce boundary condition
            ind = np.where(particleCoords[:,passUpperBound]<0)[0]
            particleCoords[ind,passUpperBound] = particleCoords[ind,passUpperBound] + 35000
                
            if 'DM' not in sim:
                # repeat for baryons
                particleCoords_bar[:,passUpperBound] = particleCoords_bar[:,passUpperBound] - 35000/2
                ind_bar = np.where(particleCoords_bar[:,passUpperBound]<0)[0]
                particleCoords_bar[ind_bar,passUpperBound] = particleCoords_bar[ind_bar,passUpperBound] + 35000
                
        if any(passLowerBound):
            particleCoords[:,passLowerBound] = particleCoords[:,passLowerBound] + 35000/2
            subhaloPos_i[passLowerBound] = subhaloPos_i[passLowerBound] + 35000/2

            ind = np.where(particleCoords[:,passLowerBound]>35000)[0]
            particleCoords[ind,passLowerBound] = particleCoords[ind,passLowerBound] - 35000
                
            if 'DM' not in sim:
                particleCoords_bar[:,passLowerBound] = particleCoords_bar[:,passLowerBound] + 35000/2                    
                ind_bar = np.where(particleCoords_bar[:,passLowerBound]>35000)[0]
                particleCoords_bar[ind_bar,passLowerBound] = particleCoords_bar[ind_bar,passLowerBound] - 35000
        # done
        ########################################################################

        particleCoords = (particleCoords - subhaloPos_i)*a #kpc/h
        paricleVels = (particleVels - np.mean(particleVels,axis=0)) * np.sqrt(a) # km/s
            
        if 'DM' not in sim:
            particleCoords_bar = (particleCoords_bar - subhaloPos_i)*a
            paricleVels_bar = (particleVels_bar - np.mean(particleVels_bar,axis=0)) * np.sqrt(a) # km/s
                
        # Measure the spin inside a given radius
        r = np.sqrt(np.sum(particleCoords**2,axis=1))
        R = outerVirialLim * Rvirial_i * a # kpc/h
        indsInBounds = np.where((r>innerVirialLim*a)&(r<=outerVirialLim*a))[0]
            
        # measured in units of h^{-1} kpc*km/s
        # *** angular momentum vector?
        J_specific = np.sum(np.cross(particleCoords[indsInBounds],particleVels[indsInBounds]),axis=0)/ len(particleCoords[indsInBounds]) 
            
        # measure lambda next
        M = len(indsInBounds) * mDM
        V = np.sqrt( const.G*(M*u.Msun)/(R*u.kpc) ).to(u.km/u.s).value
            
        lambda_prime =  J_specific / (np.sqrt(2)*V*R) # J_specific = J/M
            
        lambda_mag.append(np.sqrt(np.sum(lambda_prime**2)))
        lambda_dir.append(lambda_prime / np.sqrt(np.sum(lambda_prime**2)))
            
        # repeat for baryons
        if 'DM' not in sim:
            r_bar = np.sqrt(np.sum(particleCoords_bar**2,axis=1))
            indsInBounds = np.where((r_bar>innerVirialLim*a)&(r_bar<=outerVirialLim*a))[0]

            # measured in units of h^{-2} Msun*kpc*km/s
            J_bar = np.sum(particleMass_bar[indsInBounds].reshape(-1,1)*np.cross(particleCoords_bar[indsInBounds],
                                                                                     particleVels_bar[indsInBounds]),axis=0) 
            # measure lambda next
            M = np.sum(particleMass_bar[indsInBounds])
            V = np.sqrt( const.G*(M*u.Msun)/(R*u.kpc) ).to(u.km/u.s).value

            lambda_prime_bar =  J_bar / (np.sqrt(2)*M*V*R) # J_specific = J/M

            lambda_mag_bar.append(np.sqrt(np.sum(lambda_prime_bar**2)))
            lambda_dir_bar.append(lambda_prime_bar / np.sqrt(np.sum(lambda_prime_bar**2)))
                
        #print('Finished snap %d'%snap)
                
    print('saving results')

    #np.save(save_name+'GrNr_%d_snap_%d_99_lambda_prime.npy'%(GrNr,startSnap),(lambda_dir,lambda_mag),allow_pickle=True)
        
    if 'DM' not in sim:
        temp = save_name+'GrNr_%d_snap_%d_99_lambda_prime_baryons.npy'%(GrNr,startSnap)
        np.save(temp,lambda_dir_bar,allow_pickle=True)
        #np.save(temp,lambda_mag_bar,allow_pickle=True)

    if restart:
        completed_halos = np.append(completed_halos,GrNr)
        np.save(sim+'_halo_spin_restart.npy',completed_halos)